**Name: Huy Ngo & Po-Chien Chung**

**ID: 44681348 & 63533832**
<center><h1>Kaggle Project</h1></center>

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import mltools as ml
from importlib import reload

# Original data taken from txt files
x = np.genfromtxt("X_train.txt", delimiter = None)
y = np.genfromtxt("Y_train.txt", delimiter = None)
x_test = np.genfromtxt("X_test.txt", delimiter = None)

# Extract certain data out to be validation data
x_validation = x[10000:20000]
y_validation = y[10000:20000]

# Extract certain data out to be training data
x_train = x[0:10000]
y_train = y[0:10000]

In [ ]:
def auc(X, Y):
    soft = X
    n,d = np.atleast_2d(auc_final_prediction).shape 
    soft = soft.flatten() if n==1 else soft.T.flatten()
    
    indices = np.argsort(soft)         # sort data by score value
    Y = Y[indices]
    sorted_soft = soft[indices]

    # compute rank (averaged for ties) of sorted data
    dif = np.hstack( ([True],np.diff(sorted_soft)!=0,[True]) )
    r1  = np.argwhere(dif).flatten()
    r2  = r1[0:-1] + 0.5*(r1[1:]-r1[0:-1]) + 0.5
    rnk = r2[np.cumsum(dif[:-1])-1]

    # number of true negatives and positives
    n0,n1 = sum(Y == 0), sum(Y == 1)

    if n0 == 0 or n1 == 0:
      raise ValueError('Data of both class values not found')

    # compute AUC using Mann-Whitney U statistic
    result = (np.sum(rnk[Y == 1]) - n1 * (n1 + 1.0) / 2.0) / n1 / n0
    return result

<h2>Learner 1: Random Forest</h2>

In [4]:
# Build ensemble
size = 5
random_forest = [None for j in range(size)]
auc_train = np.array([0.0 for i in range(size)])
auc_validation = np.array([0.0 for i in range(size)])

for i in range(size):
    xBoot, yBoot = ml.utils.bootstrapData(x, y)
    random_forest[i] = ml.dtree.treeClassify(xBoot, yBoot, maxDepth = 15, minLeaf = 4)

In [7]:
# Predict Y and upload it to Kaggle
predictions_ensemble = [None] * size

for i in range(size):
    predictions_ensemble[i] = random_forest[i].predictSoft(x_test)

In [8]:
# Output prediction to a file
predict = np.mean(predictions_ensemble, axis = 0)
np.savetxt("Yhat_random_forest.txt", np.vstack( (np.arange(len(predict)) , predict[:,1]) ).T,
           '%d, %.2f',header='ID,Prob1',comments='',delimiter=',');